In [2]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

In [3]:
matchups_df = pd.read_csv("data/organized/2010/matchups.csv")
box_scores_df = pd.read_csv("data/organized/2010/box_scores.csv")
player_stats_df = pd.read_csv("data/organized/2010/player_stats.csv")
scorings_df = pd.read_csv("data/organized/2010/scorings.csv")
settings_df = pd.read_csv("data/organized/2010/settings.csv")


In [4]:
matchups_df

,matchup_id,home_team,home_team_ranking,home_team_box,away_team,away_team_ranking,away_team_box,url,neutral,winner
0,0,Pittsburgh,4,0,Rhode Island,0,1,/basketball/ncaab-boxscores.aspx?page=/data/NC...,False,True
1,1,Illinois,16,2,UC Irvine,0,3,/basketball/ncaab-boxscores.aspx?page=/data/NC...,False,True
2,2,Maryland,0,4,Seattle,0,5,/basketball/ncaab-boxscores.aspx?page=/data/NC...,False,False
3,3,Texas,25,6,Navy,0,7,/basketball/ncaab-boxscores.aspx?page=/data/NC...,False,True
4,4,Pittsburgh,4,8,Illinois-Chicago,0,9,/basketball/ncaab-boxscores.aspx?page=/data/NC...,False,True
...,...,...,...,...,...,...,...,...,...,...
5785,5785,Alabama,0,11570,Wichita St.,0,11571,/basketball/ncaab-boxscores.aspx?page=/data/NC...,True,False
5786,5786,Oregon,0,11572,Creighton,0,11573,/basketball/ncaab-boxscores.aspx?page=/data/NC...,False,True
5787,5787,Butler,8,11574,VCU,11,11575,/basketball/ncaab-boxscores.aspx?page=/data/NC...,True,True
5788,5788,Connecticut,3,11576,Kentucky,4,11577,/basketball/ncaab-boxscores.aspx?page=/data/NC...,True,True


In [5]:
games_df = pd.DataFrame()

In [6]:
settings_df['date'] = pd.to_datetime(settings_df['date'])
settings_df.dtypes

matchup_id             int64
date          datetime64[ns]
city                  object
state                 object
country              float64
arena                 object
attendance           float64
dtype: object

In [7]:
games_df['home_team'] = matchups_df['home_team'].astype('category').cat.codes
games_df['away_team'] = matchups_df['away_team'].astype('category').cat.codes
games_df['neutral'] = matchups_df['neutral'].astype('int')

games_df['day_of_week'] = settings_df['date'].dt.day_of_week

games_df['home_team_has_ranking'] = (matchups_df['home_team_ranking'] > 0).astype(int)
games_df['away_team_has_ranking'] = (matchups_df['away_team_ranking'] > 0).astype(int)

games_df['winner'] = matchups_df['winner'].astype('int')

In [8]:
games_df

,home_team,away_team,neutral,day_of_week,home_team_has_ranking,away_team_has_ranking,winner
0,241,418,0,0.0,1,0,1
1,128,536,0,0.0,1,0,1
2,169,449,0,0.0,0,0,0
3,306,339,0,0.0,1,0,1
4,241,213,0,2.0,1,0,1
...,...,...,...,...,...,...,...
5785,2,588,1,3.0,0,0,0
5786,234,109,0,4.0,0,0,1
5787,36,552,1,5.0,1,1,1
5788,67,237,1,5.0,1,1,1


In [10]:
train = games_df[settings_df['date'] < '2011-03-14']
test = games_df[settings_df['date'] > '2011-03-14']
predictors = ['neutral', 'day_of_week', 'home_team_has_ranking', 'away_team_has_ranking']

model = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)

model.fit(train[predictors], train['winner'])

RandomForestClassifier(min_samples_split=10, random_state=1)

In [11]:
preds = model.predict(test[predictors])

In [14]:
accuracy = accuracy_score(test['winner'], preds)

print(accuracy)

0.6956521739130435
